#**BulkFile Re-namer Game Project**

In [12]:
# Bulk File Renamer in Google Colab
from google.colab import files
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from datetime import datetime

class BulkFileRenamer:
    def __init__(self):
        self.uploaded_files = []
        self.rename_pattern = "file_{num:03d}{ext}"
        self.start_num = 1
        self.preview_data = []

        # Create widgets
        self.upload_btn = widgets.FileUpload(description="Upload Files", multiple=True)
        self.pattern_input = widgets.Text(
            value=self.rename_pattern,
            description="Pattern:",
            placeholder="file_{num:03d}{ext}",
            style={'description_width': 'initial'}
        )
        self.start_num_input = widgets.IntText(
            value=self.start_num,
            description="Start #:",
            style={'description_width': 'initial'}
        )
        self.preview_btn = widgets.Button(description="Preview Rename")
        self.execute_btn = widgets.Button(description="Execute Rename", button_style='success')
        self.download_btn = widgets.Button(description="Download All", button_style='info')
        self.output_area = widgets.Output()

        # Set up event handlers
        self.upload_btn.observe(self.on_upload, names='value')
        self.pattern_input.observe(self.on_pattern_change, names='value')
        self.start_num_input.observe(self.on_start_num_change, names='value')
        self.preview_btn.on_click(self.preview_rename)
        self.execute_btn.on_click(self.execute_rename)
        self.download_btn.on_click(self.download_files)

        # Display initial UI
        self.display_ui()

    def display_ui(self):
        """Display the main user interface"""
        clear_output()
        display(widgets.VBox([
            widgets.HTML("<h2>📁 Bulk File Renamer</h2>"),
            widgets.HTML("Upload files and specify renaming pattern:"),
            self.upload_btn,
            widgets.HBox([self.pattern_input, self.start_num_input]),
            widgets.HTML("<b>Available pattern variables:</b>"),
            widgets.HTML("<ul>"
                         "<li><code>{num}</code> - Sequence number</li>"
                         "<li><code>{ext}</code> - File extension</li>"
                         "<li><code>{orig}</code> - Original filename (without ext)</li>"
                         "<li><code>{date}</code> - Current date (YYYY-MM-DD)</li>"
                         "<li><code>{time}</code> - Current time (HH-MM-SS)</li>"
                         "</ul>"),
            widgets.HBox([self.preview_btn, self.execute_btn, self.download_btn]),
            self.output_area
        ]))

    def on_upload(self, change):
        """Handle file uploads"""
        self.uploaded_files = list(change['new'].values())
        with self.output_area:
            clear_output()
            print(f"📤 Uploaded {len(self.uploaded_files)} files")

    def on_pattern_change(self, change):
        """Update rename pattern"""
        self.rename_pattern = change['new']

    def on_start_num_change(self, change):
        """Update starting number"""
        self.start_num = change['new']

    def preview_rename(self, button):
        """Generate rename preview"""
        if not self.uploaded_files:
            with self.output_area:
                clear_output()
                print("⚠️ Please upload files first")
            return

        self.preview_data = []
        current_num = self.start_num
        current_date = datetime.now().strftime("%Y-%m-%d")
        current_time = datetime.now().strftime("%H-%M-%S")

        with self.output_area:
            clear_output()
            print("🔄 Rename Preview:")
            print("{:<30} {:<30}".format("Original", "New Name"))
            print("-" * 60)

            for file in self.uploaded_files:
                orig_name = file['name']
                name, ext = os.path.splitext(orig_name)

                new_name = self.rename_pattern.format(
                    num=current_num,
                    ext=ext,
                    orig=name,
                    date=current_date,
                    time=current_time
                )

                self.preview_data.append((orig_name, new_name))
                print("{:<30} → {:<30}".format(orig_name, new_name))
                current_num += 1

    def execute_rename(self, button):
        """Execute the rename operation"""
        if not self.preview_data:
            with self.output_area:
                clear_output()
                print("⚠️ Please generate preview first")
            return

        try:
            # Create temp directory
            os.makedirs("renamed_files", exist_ok=True)

            for orig_name, new_name in self.preview_data:
                # Find the uploaded file data
                file_data = next(f for f in self.uploaded_files if f['name'] == orig_name)

                # Save with new name
                with open(os.path.join("renamed_files", new_name), "wb") as f:
                    f.write(file_data['content'])

            with self.output_area:
                clear_output()
                print("✅ Successfully renamed all files!")
                print(f"Saved in folder: renamed_files/")
        except Exception as e:
            with self.output_area:
                clear_output()
                print(f"❌ Error during rename: {str(e)}")

    def download_files(self, button):
        """Download renamed files as zip"""
        if not os.path.exists("renamed_files"):
            with self.output_area:
                clear_output()
                print("⚠️ No renamed files available to download")
            return

        # Create zip archive
        zip_filename = "renamed_files.zip"
        !zip -r {zip_filename} renamed_files/

        # Download the zip file
        from google.colab import files
        files.download(zip_filename)

        with self.output_area:
            clear_output()
            print(f"📥 Downloaded: {zip_filename}")

# Start the renamer
renamer = BulkFileRenamer()